## Libraries

In [43]:
import pandas as pd
import numpy as np
import scipy.stats as stat

from math import sqrt
from mlgear.utils import show, display_columns
from surveyweights import normalize_weights


def margin_of_error(n=None, sd=None, p=None, type='proportion', interval_size=0.95):
    z_lookup = {0.8: 1.28, 0.85: 1.44, 0.9: 1.65, 0.95: 1.96, 0.99: 2.58}
    if interval_size not in z_lookup.keys():
        raise ValueError('{} not a valid `interval_size` - must be {}'.format(interval_size,
                                                                              ', '.join(list(z_lookup.keys()))))
    if type == 'proportion':
        se = sqrt(p * (1 - p)) / sqrt(n)
    elif type == 'continuous':
        se = sd / sqrt(n)
    else:
        raise ValueError('{} not a valid `type` - must be proportion or continuous')
    
    z = z_lookup[interval_size]
    return se * z


def print_pct(pct, digits=0):
    pct = pct * 100
    pct = np.round(pct, digits)
    if pct >= 100:
        if digits == 0:
            val = '>99.0%'
        else:
            val = '>99.'
            for d in range(digits - 1):
                val += '9'
            val += '0%'
    elif pct <= 0:
        if digits == 0:
            val = '<0.1%'
        else:
            val = '<0.'
            for d in range(digits - 1):
                val += '0'
            val += '1%'
    else:
        val = '{}%'.format(pct)
    return val


def calc_result(for_vote, against_vote, n, interval=0.8):
    GENERAL_POLLING_ERROR = 8.0
    TIME_SHIFT_ERROR = 4.0
    N_SIMS = 4000
    
    for_moe = margin_of_error(n=n, p=for_vote/100, interval_size=interval)
    against_moe = margin_of_error(n=n, p=against_vote/100, interval_size=interval)
    undecided = 100 - for_vote - against_vote
    mean = for_vote + undecided * 0.25
    raw_moe = for_moe * 100 + against_moe * 100
    
    allocate_undecided = undecided * 0.4
    margin = raw_moe + allocate_undecided + GENERAL_POLLING_ERROR + TIME_SHIFT_ERROR
    
    cdf_value = 0.5 + 0.5 * interval
    normed_sigma = stat.norm.ppf(cdf_value)
    sigma = margin / 100 / normed_sigma
    
    sims = np.random.normal(mean / 100, sigma, N_SIMS)
    chance_pass = np.sum([s > 0.5 for s in sims]) / N_SIMS
    low, high = np.percentile(sims, [20, 80]) * 100
    
    return {'mean': mean, 'high': high, 'low': low, 'n': n,
            'raw_moe': raw_moe, 'margin': margin, 'sigma': sigma, 'chance_pass': chance_pass}


def print_result(mean, high, low, n, raw_moe, margin, sigma, chance_pass):
    mean = np.round(mean, 1)
    first = np.round(high, 1)
    second = np.round(low, 1)
    sigma = np.round(sigma * 100, 1)
    raw_moe = np.round(raw_moe, 1)
    margin = np.round(margin, 1)
    chance_pass = print_pct(chance_pass, 1)
    if second < first:
        _ = first
        first = second
        second = _
    print(('Result {} (80% CI: {} to {}) (N={}) (raw_moe={}pts, margin={}pts, '
           'sigma={}pts) ({} likely to pass)').format(mean,
                                                      first,
                                                      second,
                                                      n,
                                                      raw_moe,
                                                      margin,
                                                      sigma,
                                                      chance_pass))
    print(('{} (80% CI: {} to {}) ({})').format(mean,
                                                first,
                                                second,
                                                chance_pass))
    print('-')

## Load Processed Data

In [44]:
survey = pd.read_csv('repsonses_processed_national_weighted.csv').fillna('Not presented')
ca_national_survey = pd.read_csv('repsonses_processed_ca_weighted.csv').fillna('Not presented')

/Users/peterhurford/.virtualenvs/dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,40,41,42,43,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## California Propositions

In [73]:
options = ['Vote for / Support', 'Vote against / Oppose', 'Don’t know / Undecided']
ca_measures = [c for c in survey.columns if 'vote_measure_ca_' in c and 'meta' not in c]

for measure in ca_measures:
    survey_ = survey.loc[survey[measure].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
    print('## {} NATIONAL UNWEIGHTED ##'.format(measure))
    n = len(survey_)
    votes = survey_[measure].value_counts(normalize=True) * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=n))
    
    print('## {} NATIONAL WEIGHTED ##'.format(measure))
    weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=weighted_n))

    print('## {} NATIONAL WEIGHTED + LV ##'.format(measure))
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    if measure != 'vote_measure_100pct_income_tax':
        survey_ = survey.loc[survey[measure].isin(options)].copy()
        survey_['weight'] = normalize_weights(survey_['weight'])
        survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
        survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

        print('## {} NATIONAL CORRECTED + WEIGHTED + LV ##'.format(measure))
        lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
        votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
        votes = votes[options] * (100 / votes[options].sum())
        print(votes)
        print_result(**calc_result(for_vote=votes['Vote for / Support'],
                                   against_vote=votes['Vote against / Oppose'],
                                   n=lv_weighted_n))
        
    survey_ = survey.loc[ca_national_survey[measure].isin(options)].copy()
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
    print('## {} NATIONAL CA-WEIGHTED ##'.format(measure))
    weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=weighted_n))

    print('## {} NATIONAL CA-WEIGHTED + LV ##'.format(measure))
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    if measure != 'vote_measure_100pct_income_tax':
        survey_ = ca_national_survey.loc[survey[measure].isin(options)].copy()
        survey_['weight'] = normalize_weights(survey_['weight'])
        survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
        survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])

        print('## {} NATIONAL CORRECTED + CA-WEIGHTED + LV ##'.format(measure))
        lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
        votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
        votes = votes[options] * (100 / votes[options].sum())
        print(votes)
        print_result(**calc_result(for_vote=votes['Vote for / Support'],
                                   against_vote=votes['Vote against / Oppose'],
                                   n=lv_weighted_n))

    survey_ = ca_national_survey.loc[survey[measure].isin(options)].copy()
    survey_ = survey_.loc[survey['state'] == 'California']
    survey_['weight'] = normalize_weights(survey_['weight'])
    survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
    survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
    
    print('## {} CALI UNWEIGHTED ##'.format(measure))
    n = len(survey_)
    votes = survey_[measure].value_counts(normalize=True) * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=n))
    
    print('## {} CALI CA-WEIGHTED ##'.format(measure))
    weighted_n = int(np.round(survey_['weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=weighted_n))

    print('## {} CALI CA-WEIGHTED + LV ##'.format(measure))
    lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
    votes = survey_[measure].value_counts(normalize=True) * survey.groupby(measure)['lv_weight'].mean() * 100
    votes = votes[options] * (100 / votes[options].sum())
    print(votes)
    print_result(**calc_result(for_vote=votes['Vote for / Support'],
                               against_vote=votes['Vote against / Oppose'],
                               n=lv_weighted_n))
    
    if measure != 'vote_measure_100pct_income_tax':
        survey_ = ca_national_survey.loc[survey[measure].isin(options)].copy()
        survey_ = survey_.loc[survey['state'] == 'California']
        survey_['weight'] = normalize_weights(survey_['weight'])
        survey_['rv_weight'] = normalize_weights(survey_['rv_weight'])
        survey_['lv_weight'] = normalize_weights(survey_['lv_weight'])
        
        print('## {} CALI CORRECTED + CA-WEIGHTED + LV ##'.format(measure))
        lv_weighted_n = int(np.round(survey_['lv_weight'].apply(lambda w: 1 if w > 1 else w).sum()))
        votes = survey_[measure].value_counts(normalize=True) * survey_.groupby(measure)['lv_weight'].mean() * 100
        votes = votes[options] * (100 / votes[options].sum())
        print(votes)
        print_result(**calc_result(for_vote=votes['Vote for / Support'],
                                   against_vote=votes['Vote against / Oppose'],
                                   n=lv_weighted_n))

    print('-')

## vote_measure_ca_prop16_affirmative_action NATIONAL UNWEIGHTED ##
Vote for / Support        42.950237
Vote against / Oppose     38.447867
Don’t know / Undecided    18.601896
Name: vote_measure_ca_prop16_affirmative_action, dtype: float64
Result 47.6 (80% CI: 32.7 to 62.2) (N=1688) (raw_moe=3.1pts, margin=22.5pts, sigma=17.6pts) (44.9% likely to pass)
47.6 (80% CI: 32.7 to 62.2) (44.9%)
-
## vote_measure_ca_prop16_affirmative_action NATIONAL WEIGHTED ##
Vote for / Support        36.724107
Vote against / Oppose     45.900819
Don’t know / Undecided    17.375074
dtype: float64
Result 41.1 (80% CI: 25.5 to 56.4) (N=691) (raw_moe=4.8pts, margin=23.7pts, sigma=18.5pts) (31.7% likely to pass)
41.1 (80% CI: 25.5 to 56.4) (31.7%)
-
## vote_measure_ca_prop16_affirmative_action NATIONAL WEIGHTED + LV ##
Vote for / Support        38.935427
Vote against / Oppose     48.836327
Don’t know / Undecided    12.228246
dtype: float64
Result 42.0 (80% CI: 27.8 to 56.8) (N=667) (raw_moe=4.9pts, margin=21.8p

Result 58.6 (80% CI: 42.9 to 74.3) (N=686) (raw_moe=4.6pts, margin=23.8pts, sigma=18.6pts) (68.8% likely to pass)
58.6 (80% CI: 42.9 to 74.3) (68.8%)
-
## vote_measure_ca_prop14_stem_cell CALI UNWEIGHTED ##
Vote for / Support        65.988701
Vote against / Oppose     17.514124
Don’t know / Undecided    16.497175
Name: vote_measure_ca_prop14_stem_cell, dtype: float64
Result 70.1 (80% CI: 55.7 to 84.5) (N=885) (raw_moe=3.7pts, margin=22.3pts, sigma=17.4pts) (87.6% likely to pass)
70.1 (80% CI: 55.7 to 84.5) (87.6%)
-
## vote_measure_ca_prop14_stem_cell CALI CA-WEIGHTED ##
Vote for / Support        51.206049
Vote against / Oppose     31.736339
Don’t know / Undecided    17.057612
dtype: float64
Result 55.5 (80% CI: 38.8 to 71.3) (N=374) (raw_moe=6.4pts, margin=25.2pts, sigma=19.7pts) (60.4% likely to pass)
55.5 (80% CI: 38.8 to 71.3) (60.4%)
-
## vote_measure_ca_prop14_stem_cell CALI CA-WEIGHTED + LV ##
Vote for / Support        55.274518
Vote against / Oppose     28.959846
Don’t know / U

Result 49.4 (80% CI: 30.4 to 67.1) (N=355) (raw_moe=6.6pts, margin=27.7pts, sigma=21.6pts) (47.8% likely to pass)
49.4 (80% CI: 30.4 to 67.1) (47.8%)
-
-
## vote_measure_ca_prop23_dialysis NATIONAL UNWEIGHTED ##
Vote for / Support        52.472368
Vote against / Oppose     25.421757
Don’t know / Undecided    22.105876
Name: vote_measure_ca_prop23_dialysis, dtype: float64
Result 58.0 (80% CI: 42.0 to 73.6) (N=1719) (raw_moe=2.9pts, margin=23.7pts, sigma=18.5pts) (66.6% likely to pass)
58.0 (80% CI: 42.0 to 73.6) (66.6%)
-
## vote_measure_ca_prop23_dialysis NATIONAL WEIGHTED ##
Vote for / Support        53.493754
Vote against / Oppose     29.181066
Don’t know / Undecided    17.325180
dtype: float64
Result 57.8 (80% CI: 42.2 to 73.3) (N=724) (raw_moe=4.5pts, margin=23.5pts, sigma=18.3pts) (66.2% likely to pass)
57.8 (80% CI: 42.2 to 73.3) (66.2%)
-
## vote_measure_ca_prop23_dialysis NATIONAL WEIGHTED + LV ##
Vote for / Support        55.194511
Vote against / Oppose     28.691687
Don’t kno

Vote for / Support        60.237468
Vote against / Oppose     28.385574
Don’t know / Undecided    11.376958
dtype: float64
Result 63.1 (80% CI: 49.1 to 77.0) (N=668) (raw_moe=4.7pts, margin=21.2pts, sigma=16.5pts) (77.9% likely to pass)
63.1 (80% CI: 49.1 to 77.0) (77.9%)
-
## vote_measure_ca_prop17_felon_vote NATIONAL CA-WEIGHTED + LV ##
Vote for / Support        62.707959
Vote against / Oppose     30.668603
Don’t know / Undecided     6.623438
dtype: float64
Result 64.4 (80% CI: 51.3 to 77.8) (N=649) (raw_moe=4.7pts, margin=19.4pts, sigma=15.1pts) (82.5% likely to pass)
64.4 (80% CI: 51.3 to 77.8) (82.5%)
-
## vote_measure_ca_prop17_felon_vote NATIONAL CORRECTED + CA-WEIGHTED + LV ##
Vote for / Support        62.707959
Vote against / Oppose     30.668603
Don’t know / Undecided     6.623438
dtype: float64
Result 64.4 (80% CI: 51.8 to 77.0) (N=649) (raw_moe=4.7pts, margin=19.4pts, sigma=15.1pts) (83.6% likely to pass)
64.4 (80% CI: 51.8 to 77.0) (83.6%)
-
## vote_measure_ca_prop17_felon